In [144]:
from polychrom.hdf5_format import HDF5Reporter
from polychrom.polymerutils import fetch_block
from polychrom import forcekits, forces, simulation, starting_conformations
from polychrom.hdf5_format import HDF5Reporter
import numpy as np
import openmm as mm
import pickle 
import math
from typing import Dict, List, Tuple, TypedDict

import numpy as np

In [ ]:
class MoleculeStructure(TypedDict):
    positions: np.ndarray  # (N, 3) array of particle positions
    bonds: List[Tuple[int, int, float, float]]  # list of bond tuples (i, j, ℓ₀, Δℓ_kT)
    angles: List[Tuple[int, int, int, float, float]]  # list of angle tuples (i, j, k, θ₀, k_ang)
    index_dict: Dict[Tuple[int, int], int]  # maps global index to (layer, vertex_id)


class MoleculeStructureWithNames(TypedDict):
    positions: np.ndarray  # (N, 3) array of particle positions
    bonds: List[Tuple[int, int, float, float]]  # list of bond tuples (i, j, ℓ₀, Δℓ_kT)
    angles: List[Tuple[int, int, int, float, float]]  # list of angle tuples (i, j, k, θ₀, k_ang)
    index_dict: Dict[Tuple[str, int, int], int]  # maps (name, layer, vertex_id) to global index


def concatenate_molecule_structures(
    structures: List[MoleculeStructure],
    names: List[str] | None = None,
    extra_bonds: List[Tuple[int, int, float, float]] | None = None,
    extra_angles: List[Tuple[int, int, int, float, float]] | None = None,
) -> MoleculeStructureWithNames:
    """
    Concatenate multiple MoleculeStructure dictionaries into one.
    Ensures indices don't overlap and are consecutive.

    Parameters
    ----------
    structures : List[MoleculeStructure]
        List of MoleculeStructure dictionaries to concatenate.
    names : List[str] | None, optional
        List of names corresponding to each structure, used for index lookup
    Returns
    -------
    MoleculeStructure
        A single MoleculeStructure containing concatenated positions, bonds, angles, and index_dict.
    """
    # making sure indices are unique and consecutive
    count_array = np.zeros(sum(len(s["positions"]) for s in structures), dtype=int)
    for s in structures:
        for idx in s["index_dict"].values():
            count_array[idx] += 1
    if np.any(count_array > 1):
        raise ValueError("Indices in structures are not unique. Please ensure each structure has unique indices.")
    if np.any(count_array < 1):
        raise ValueError("Indices in structures are not consecutive. Please ensure indices are consecutive.")

    if names is None:
        names = [f"structure_{i}" for i in range(len(structures))]
    total_positions = np.concatenate([s["positions"] for s in structures], axis=0)
    total_bonds = [bond for s in structures for bond in s["bonds"]]
    total_angles = [angle for s in structures for angle in s["angles"]]

    # Update index_dict to account for global indices
    index_dict = {}
    for s, name in zip(structures, names):
        for (layer, vid), idx in s["index_dict"].items():
            index_dict[(name, layer, vid)] = idx

    if extra_bonds is not None:
        total_bonds.extend(extra_bonds)
    if extra_angles is not None:
        total_angles.extend(extra_angles)

    return {
        "positions": total_positions,
        "bonds": total_bonds,
        "angles": total_angles,
        "index_dict": index_dict,
    }


def build_octagonal_prism(
    center_pos: np.ndarray | Tuple[float, float, float],
    start_index: int,
    n_rows: int,
    bond_dev: float = 0.02,
    angle_force_k: float = 60.0,
) -> MoleculeStructure:
    """
    Construct a coarse-grained octagonal prism with a central axle.

    Parameters
    ----------
    center_pos : (3,) array_like
        XYZ coordinates of the centre of the **lowest (row 0)** layer.
    start_index : int
        Global particle index assigned to the first vertex in row 0.
    n_rows : int
        Number of layers stacked in +Z.
    bond_dev : float, default 0.05
        Δℓ at which a harmonic bond reaches 1 kT.
    angle_force_k : float, default 20.0
        Harmonic angle force constant (kT / rad²).

    Returns
    -------
    MoleculeStructure
        A dictionary containing:
        - `positions`: (N, 3) array of particle positions.
        - `bonds`: List of bond tuples (i, j, ℓ₀, Δℓ_kT).
        - `angles`: List of angle tuples (i, j, k, θ₀, k_ang).
        - `index_dict`: Maps (layer, vertex_id) to global index.

    Notes
    -----
    Geometry:
    - Side length of octagon = 1 (simulation length unit).
    - Layer spacing along Z = 1.
    - Octagon is rotated so that its edges are axis-aligned; vertex 0 is at
      (x>0, y>0) closest to X axis and angles increase CCW in the XY-plane.

    Bond topology:
    - Ring edges, radial spokes to axle, vertical columns, and axial column.

    Angle topology:
    - For each vertex in every inter-layer pair, impose four π/2 angles:
      (next_vertex, current_vertex, current_vertex↑)
      (prev_vertex, current_vertex, current_vertex↑)
      (next_vertex, current_vertex, current_vertex↓)
      (prev_vertex, current_vertex, current_vertex↓)

    - This ensures torsional rigidity between layers.
    """
    center = np.asarray(center_pos, dtype=float)
    assert center.shape == (3,), "center_pos must be length-3."

    n_vertices = 8
    pts_per_layer = n_vertices + 1
    total_pts = pts_per_layer * n_rows
    positions = np.empty((total_pts, 3), dtype=float)
    bonds: List[Tuple[int, int, float, float]] = []
    angles: List[Tuple[int, int, int, float, float]] = []
    index_dict: Dict[Tuple[int, int], int] = {}

    # Pre-compute vertex coordinates in the XY-plane
    side = 1.0
    R = side / (2.0 * math.sin(math.pi / 8.0))  # circumscribed radius
    offset = math.pi / 8.0  # edges parallel to axes
    xy_coords = np.column_stack(
        (
            R * np.cos(np.arange(n_vertices) * 2.0 * math.pi / n_vertices + offset),
            R * np.sin(np.arange(n_vertices) * 2.0 * math.pi / n_vertices + offset),
        )
    )  # shape (8, 2)

    def gid(layer: int, vid: int) -> int:
        """Global particle index helper (vid 0-7 vertices, 8 axle)."""
        return start_index + layer * pts_per_layer + vid

    # Build layers
    for layer in range(n_rows):
        z = center[2] + layer * 1.0
        base_global = start_index + layer * pts_per_layer

        # Vertices
        for vid in range(n_vertices):
            idx = base_global + vid
            positions[idx - start_index] = np.array([center[0] + xy_coords[vid, 0], center[1] + xy_coords[vid, 1], z])
            index_dict[(layer, vid)] = idx

        # Axle
        axle_idx = base_global + n_vertices
        positions[axle_idx - start_index] = np.array([center[0], center[1], z])
        index_dict[(layer, 8)] = axle_idx

        # In-layer bonds: ring edges
        for vid in range(n_vertices):
            bonds.append((gid(layer, vid), gid(layer, (vid + 1) % n_vertices), side, bond_dev))

        # In-layer bonds: spokes to axle
        for vid in range(n_vertices):
            bonds.append((gid(layer, vid), axle_idx, R, bond_dev))

        # Vertical bonds (skip on last layer)
        if layer < n_rows - 1:
            next_layer = layer + 1
            axle_next = gid(next_layer, 8)
            bonds.append((axle_idx, axle_next, 1.0, bond_dev))  # axle continuity
            for vid in range(n_vertices):
                bonds.append((gid(layer, vid), gid(next_layer, vid), 1.0, bond_dev))

    # Inter-layer π/2 angles for torsional rigidity
    for layer in range(n_rows - 1):
        next_layer = layer + 1
        for vid in range(n_vertices):
            v_curr = gid(layer, vid)
            v_next = gid(layer, (vid + 1) % n_vertices)
            v_prev = gid(layer, (vid - 1) % n_vertices)
            v_up = gid(next_layer, vid)

            angles.append((v_next, v_curr, v_up, math.pi / 2.0, angle_force_k))
            angles.append((v_prev, v_curr, v_up, math.pi / 2.0, angle_force_k))

            # Now repeat in the "other direction"
            v_curr = gid(next_layer, vid)
            v_next = gid(next_layer, (vid + 1) % n_vertices)
            v_prev = gid(next_layer, (vid - 1) % n_vertices)
            v_down = gid(layer, vid)

            angles.append((v_next, v_curr, v_down, math.pi / 2.0, angle_force_k))
            angles.append((v_prev, v_curr, v_down, math.pi / 2.0, angle_force_k))

    # return the constructed structure
    return MoleculeStructure(positions=positions, bonds=bonds, angles=angles, index_dict=index_dict)



from openmm import CustomCompoundBondForce

class AngleCouplingForce(): 
    """
    Implements a force that couples the rotation of two hinges with bending of the hinges. 
    A part of a larger simulation of cohesin (SMC1/3 are each two prisms) in which imposing a torque between
    the two cohesin "heads" leads to unbending of the "door-hinge" in the middle of both SMC1/3 double-prisms. 

    Particles are numbered how it is shown below. 
    * 1,2,4,3 is already a nearly perfect planar square 
    * 3,4,5,6 are enforced to be planar but can pivot to the sides - bonds 3-5 and 4-6 can be made a hair longer to encourage bending
    * 5,6,8,7 is a perfect planar square

    # In this application the 8 particles are a "hinge" between two hexagonal prisms with a central axis each.
    # The axis of both prisms is "behind" the picture
    # And the hinge bends such that 1-2 / 7-8 are "towards" the viewer
    # But it can't bend with the junction towards the viewer, 
    # as that would lead to steric overlap of the bodies of the prisms (which are behind the picture)
    # vid1  vid2
    #   --prism1--  ("top prism")  -----> direction of rotational force
    # | |        | |
    # --1--------2--
    # | |        | |
    # --3--------4--
    #   |        | actual junction between the two prisms
    # --5--------6--
    # | |        | |
    # --7--------8--
    # | |        | |
    #   --prism2--  ("bottom prism")
    

    There are 3 major degrees of freedom left here:
    -- Bending over the 3-4 line (i.e. angle 1-3-5 = angle 2-4-6)
    -- Bending over the 5-6 line (i.e. angle 3-5-7 = angle 4-6-8)
    -- Rotation of one of the prisms making the torque between the two prisms
       If the top prism is "pushed" to the right by rotation, that would lead to:
       -- angles 3-5-6 6-4-3 becoming less than 90 degrees 
       -- angles 5-6-4 and 4-3-5 becoming more than 90 degrees 
    
    To couple the rotation of the prisms with the bending of the hinges we will use something akin to: 
    -- Calculate the "torque" as the difference between the 4 angles
      -- likely T = sin(356) + sin(643) - sin(564) - sin(435)
      -- Protected from angle = 0 or 180 (steric overlap of 1-5 or 2-6 pairs) -> sin is OK? 
    -- Calculate the overall bending of the hinges- force has to be strongest at 90 degree angles
      -- likely B = cos(135) + cos(246) + cos(357) + cos(468)
      -- Or maybe we should get more "reach" and grab 137, 248, 157, 268?
        -- Those can be between 30-ish and 180
        -- Those directly exert torque on both prisms, as opposed to grabbing one prism by the edge
           (e.g. grabbing the edge 3-4 and pushing it against the second row of another prism, 7-8
           which only creates torque on the second prism, not the first one)
        -- the fact taht they are of different lengths is not a problem probably 
    -- Pair them with something like k ( x * T - B )² where x is a coupling constant

    """

    force = CustomCompoundBondForce(8, ...) 

def connect_prisms(
    bonds: List[Tuple[int, int, float, float]],
    angles: List[Tuple[int, int, int, float, float]],
    dihedrals: List[Tuple[int, int, int, int, float]],
    top_prism: MoleculeStructure,
    bottom_prism: MoleculeStructure,
    vid1: int = 1,
    vid2: int = 2,
    bond_d=0.03,
    angle_force_k=50.0,
    dihedral_force_k=60.0,
) -> List[Tuple[int, int, int, int]]:
    """
    Connect two prisms by adding bonds and angles between specified vertices.
    Connects the bottom of the top prism to the top of the bottom prism.

    Returns a list of indices for the special force that couples rotation of the prisms to the closing of the hinge
    """

    assert abs(vid1 - vid2) == 1 or abs(vid1 - vid2) == 7, "vid1 and vid2 must be adjacent vertices (differ by 1)."
    assert (vid1 < vid2) or (vid1 == 7 and vid2 == 0), "vid1 must be less than vid2 or wrap around from 7 to 0."

    idict1 = top_prism["index_dict"]
    idict2 = bottom_prism["index_dict"]

    bottom = max(k[0] for k in idict2.keys())

    # we have 4 particles actually making the bond. Let's number them i,j,k,l
    # Let's also add two particles above and below the junction, which we will call i0,j0,k0,l0

    # vid1  vid2
    # --prism1--  ("top prism")
    # i0------j0
    # |        |
    # i--------j
    # |        | actual junction between the two prisms
    # l--------k
    # |        |
    # l0------k0
    # --prism2--  ("bottom prism")

    i, j, k, l = idict1[(bottom, vid1)], idict1[(bottom, vid2)], idict2[(0, vid2)], idict2[(0, vid1)]
    i0, j0, k0, l0 = idict1[(bottom - 1, vid1)], idict1[(bottom - 1, vid2)], idict2[(1, vid2)], idict2[(1, vid1)]

    # Connect the prisms with harmonic bonds
    bonds.append((i, l, 1.5, bond_d))  # i to l
    bonds.append((j, k, 1.5, bond_d))  # j to k

    # We add two dihedrals to keep the (i,j,k,l) junction planar
    # avoid "folding" along the two diagonals
    dihedrals.append((i, j, l, k, dihedral_force_k))  # i-j-l-k - along the jl diagonal
    dihedrals.append((j, i, k, l, dihedral_force_k))  # j-i-k-l - along the ik diagonal

    return []

TypeError: Wrong number or type of arguments for overloaded function 'new_CustomCompoundBondForce'.
  Possible C/C++ prototypes are:
    OpenMM::CustomCompoundBondForce::CustomCompoundBondForce(int,std::string const &)
    OpenMM::CustomCompoundBondForce::CustomCompoundBondForce(OpenMM::CustomCompoundBondForce const &)


In [ ]:
extra_bonds = []
extra_angles = []
dihedrals = []
N_ROWS = 6  # Number of rows in each prism

# Building the cohesin module 
# prism1 and prism2 are one SMC leg, prism3 and prism4 are the other SMC leg.
# It is made of two prisms because SMC has been shown to "bend in half" in the middle of the coiled coil. 
# Connection is sometimes referred to as "door-hinge" to differentiate it from the "hinge" of the SMC complex.
# the "complex-hinge" the connection between prism2 and prism4, at the 'top' of them (last row)
# Cohesin "heads" are the bottoms of prism1 and prism3. 


# put prism2 above prism2 and connect them
prism1 = build_octagonal_prism(center_pos=(0.0, 0.0, 0.0), start_index=0, n_rows=N_ROWS)
prism2 = build_octagonal_prism((0.0, 0.0, float(N_ROWS)), max(prism1["index_dict"].values()) + 1, N_ROWS)
connect_prisms(bonds=extra_bonds, angles=extra_angles, dihedrals=dihedrals, top_prism=prism1, bottom_prism=prism2)

# add two more prisms shifted to the right (+x) by 1 + sqrt(4 + 2 * sqrt(2)) - octagon's long diagonal
x_off = 1.0 + math.sqrt(4 + 2 * math.sqrt(2))
prism3 = build_octagonal_prism((x_off, 0.0, 0.0), max(prism2["index_dict"].values()) + 1, N_ROWS)
prism4 = build_octagonal_prism((x_off, 0.0, float(N_ROWS)), max(prism3["index_dict"].values()) + 1, N_ROWS)
connect_prisms(bonds=extra_bonds, angles=extra_angles, dihedrals=dihedrals, top_prism=prism3, bottom_prism=prism4)

id1, id2, id3, id4 = [i["index_dict"] for i in (prism1, prism2, prism3, prism4)]

c1 = 7 
c2 = 4
# a square formed by the top row and bottom row is (i,j,k,l) 
i,j,k,l = id2[(N_ROWS - 1, c1)], id4[(N_ROWS - 1, c2)],  id4[(N_ROWS - 2, c2)], id2[(N_ROWS - 2, c1)]

# connect the very tops of the prisms at position (0,4) and a more flexible bond at next row
extra_bonds.append((i, j, 1.0, 0.05))  # top of prism1 to top of prism2
extra_bonds.append((k, l, 1.0, 0.15))

# add dihedrals to keep the top of the prisms planar
dihedrals.append((i,j,l,k, 60.0))  # i-j-l-k - along the jl diagonal
dihedrals.append((j,i,k,l, 60.0))  # j-i-k-l - along the ik diagonal

# add right angles to keep the bonds perpendicular to prisms
extra_angles.append((id2[(N_ROWS - 1, c1)], id2[(N_ROWS - 2, c1)], id4[(N_ROWS - 2, c2)], math.pi / 2.0, 50.0))
extra_angles.append((id2[(N_ROWS-2, c1)],  id2[(N_ROWS - 1, c1)], id4[(N_ROWS - 1, c2)], math.pi / 2.0, 50.0))
extra_angles.append((id4[(N_ROWS - 1, c2)], id4[(N_ROWS - 2, c2)], id2[(N_ROWS - 2, c1)], math.pi / 2.0, 50.0))
extra_angles.append((id4[(N_ROWS - 2, c2)], id4[(N_ROWS - 1, c2)], id2[(N_ROWS - 1, c1)], math.pi / 2.0, 50.0))

# connect the two heads with a large bond of length 5, conntcting the centers of the prisms 
extra_bonds.append((id1[(0, 8)], id3[(0, 8)], 5.0, 0.2))

# connect the positions along the head to induce torque motion
#extra_bonds.append((id1[(0, 5)], id3[(0,6)], 1.5, 0.3))

# concatenate the two structures
combined_structure = concatenate_molecule_structures([prism1, prism2, prism3, prism4], extra_bonds=extra_bonds)


N = combined_structure["positions"].shape[0]
positions = combined_structure["positions"]
bonds = combined_structure["bonds"]
angles = combined_structure["angles"]

folded = pickle.load(open("folded_conf.pkl", "rb"   ))
positions = folded 

reporter = HDF5Reporter(folder="trajectory", max_data_length=5, overwrite=True)
sim = simulation.Simulation(
    platform="OpenCL",
    integrator="variableLangevin",
    error_tol=0.001,
    GPU="0",
    collision_rate=0.05,
    N=N,
    save_decimals=2,
    PBCbox=False,
    reporters=[reporter],
)

sim.set_data(positions, center=True)  # loads a polymer, puts a center of mass at zero

sim.add_force(forces.spherical_confinement(sim, r=30, k=1))

# nonbonded force
rep_force = forces.polynomial_repulsive(sim, trunc=50, radiusMult=1)

sim.add_force(rep_force)  

bond_pos = [(i[0], i[1]) for i in bonds]  # convert to positions
bond_lengths = [float(i[2]) for i in bonds]  # convert to lengths
bond_devs = [float(i[3]) for i in bonds]  # convert to widths

angle_pos = [(i[0], i[1], i[2]) for i in angles]  # convert to angle force triplets
angle_thetas = [float(i[3]) for i in angles]  # convert to angle force default angle
angle_ks = [float(i[4]) for i in angles]  # convert angle force spring constants

bond_force = forces.harmonic_bonds(sim, bond_pos, bondLength=bond_lengths, bondWiggleDistance=bond_devs)
angle_force = forces.angle_force(sim, angle_pos, theta_0=angle_thetas, k=angle_ks)

# add bond and angle forces
sim.add_force(bond_force)
sim.add_force(angle_force)


# "Flattening" dihedral force 
# To flatten a square (or any parallelogram-like thingie) apply dihedral force to the diagonals 
# i.e. if (i,j,k,l) are the vertices of a square, apply dihedral force to (i,j,l,k) and (j,i,k,l)

dihedral_force = mm.CustomTorsionForce("k * kT * (1 + cos(theta))")
dihedral_force.name = "dihedral_force" # type: ignore - this is polychrom's old convendion
dihedral_force.addPerTorsionParameter("k")
dihedral_force.addGlobalParameter("kT", sim.kT)

# Note that theta_0 is no longer needed for this specific case.
# The 'k' here is the force constant for the cosine potential.
for i in dihedrals:
    # Assuming i is a tuple/list: (atom1, atom2, atom3, atom4, k_value)
    dihedral_force.addTorsion(i[0], i[1], i[2], i[3], [i[4]])

sim.add_force(dihedral_force)

sim.eK_critical = 1000

sim.local_energy_minimization()


for block in range(150):  # Do 10 blocks
    sim.do_block(1 if block == 0 else 1500, save_extras={"bonds":bonds})  # Of 100 timesteps each. Data is saved automatically.
sim.print_stats()  # In the end, print very simple statistics

reporter.dump_data()  # always need to run in the end to dump the block cache to the disk


INFO:root:Performing local energy minimization
INFO:root:adding force spherical_confinement 0
INFO:root:adding force polynomial_repulsive 1
INFO:root:adding force harmonic_bonds 2
INFO:root:adding force angle 3
INFO:root:adding force dihedral_force 4
INFO:root:Particles loaded. Potential energy is 7.277137
INFO:root:before minimization eK=1.5892777888017506, eP=7.277137124973512, time=0.0 ps
INFO:root:Particles loaded. Potential energy is 2.577202
INFO:root:after minimization eK=1.5892777888017506, eP=0.0005097886257154975, time=0.0 ps
INFO:root:block    0 pos[1]=[-1.8 3.8 -2.9] dr=0.01 t=0.0ps kin=2.18 pot=2.07 Rg=3.597 SPS=452 dt=14.3fs dx=4.72pm 
INFO:root:block    1 pos[1]=[-1.7 4.3 -2.7] dr=0.50 t=14.8ps kin=55.41 pot=71.31 Rg=3.657 SPS=4514 dt=6.7fs dx=11.21pm 
INFO:root:block    2 pos[1]=[-1.4 3.4 -2.9] dr=0.79 t=26.1ps kin=35.00 pot=40.06 Rg=3.843 SPS=4860 dt=8.2fs dx=10.82pm 
INFO:root:block    3 pos[1]=[-0.6 3.2 -2.2] dr=0.84 t=39.4ps kin=18.53 pot=21.54 Rg=4.045 SPS=4818 dt=


 Statistics: number of particles: 216

Statistics for particle position
     mean position is:  [ 2.0027268   4.17758206 -7.03963871]   Rg =  4.44885
     median bond size is  1.0055684799270581
     three shortest/longest (<10)/ bonds are  [0.97641685 0.9767067  0.97920552]    [1.71641298 2.40630111 2.99489223]
longest 10 bonds are [ 1.6552927   1.67768883  1.67958323  1.68763456  1.70165116  1.70554314
  1.71641298  2.40630111  2.99489223 12.01885881]
     95 percentile of distance to center is:    6.422869126703202
     density of closest 95% monomers is:    0.18488493654480792
     density of the 5% closest to CoM monomers is:    0.3682130842550194
     min/median/mean/max coordinates are: 
     x: -4.24, 2.08, 2.00, 8.58
     y: 1.32, 4.20, 4.18, 7.12
     z: -11.26, -7.32, -7.04, -1.14

Statistics for velocities:
     mean kinetic energy is:  1.5268584664962976 should be: 1.5
     fastest particles are (in kT):  [6.06440611 6.39026433 6.96180359 8.33605704 9.8814308 ]

Statistic

In [55]:
data = fetch_block("trajectory",9)

In [56]:
angles

[(0, 1, 2, 1.5707963267948966, 5)]